In [4]:
import openai
import pandas as pd
import re,json
# Set your OpenAI API key here
API_Key = "APIkey"
# Load your dataset (replace with your actual file path)
original_columns = ['Primary Key', 'Order Date', 'Product Category', 'Complaint', 'Cause', 'Correction']

response_schema = {
    "type": "json_schema",
    "json_schema": {
        "name": "maintenance_classification",
        "schema": {
            "type": "object",
            "properties": {
                "root_cause": {
                    "type": "array",
                    "items": { "type": "string" },
                    "description": "List of root causes identified"
                },
                "symptom_conditions": {
                    "type": "array",
                    "items": { "type": "string" },
                    "description": "List of symptom conditions (e.g. Won't stay open, Oil leak)"
                },
                "symptom_components": {
                    "type": "array",
                    "items": { "type": "string" },
                    "description": "List of components associated with each symptom condition"
                },
                "fix_conditions": {
                    "type": "array",
                    "items": { "type": "string" },
                    "description": "List of fix conditions (e.g. Installed, Tightened)"
                },
                "fix_components": {
                    "type": "array",
                    "items": { "type": "string" },
                    "description": "List of components that were fixed"
                },
                "estimated_time_to_resolve_hours": {
                    "type": "number",
                    "description": "Estimated total time required to resolve all the issues (in hours)"
                }
            },
            "required": [
                "root_cause",
                "symptom_conditions",
                "symptom_components",
                "fix_conditions",
                "fix_components",
                "estimated_time_to_resolve_hours"
            ],
            "additionalProperties": False
        }
    }
}

df = pd.read_excel('input.xlsx')
# Define the Chain-of-Thought + Instructional Prompt
def generate_prompt(complaint, cause, correction):
    return f"""
You are a highly skilled service data analyst.

Your task is to classify maintenance records based on the complaint, cause, and correction.  
Analyze the inputs step by step and assign the correct Root Cause, Symptom Condition, Symptom Component, Fix Condition, and Fix Component.  
Use the taxonomy table below for reference when tagging. If a field is not applicable, use "None".

IMPORTANT: If there are multiple issues in the complaint, cause, or correction, make sure to capture ALL of them in the output.
For example, if there are multiple symptoms, list them all in Symptom Condition 1, 2, 3 etc.
If there are multiple fixes, list them all in Fix Condition 1, 2, 3 etc.

Also estimate how much time (in hours) is typically needed to resolve ALL the issues based on the correction.

=== REFERENCE TAXONOMY ===

Root Cause | Symptom Condition | Symptom Component | Fix Condition | Fix Component
-----------|-------------------|-------------------|----------------|---------------
Not Tightened | Loose | Cab P Clip | Retightened | Cab P Clip  
Not Installed | Won't stay open | Fuel Door | Installed | Gas Strut  
Not Mentioned | Crushed | Compressor Pressure Line | Replaced | Braided Steel  
Loosened | Oil Running | Not Mentioned | Topped Off | O-Ring  
Not Included | Missing | Vector | Not Mentioned | Vector  
Out of Fitting | Oil Dripping | Coupler | Cleaned Out | Coupler  
Blown | Oil Leak | Mount SVM Sign | Reseted | Brackets  
Poor Material | Broke | Harness | Repaired | Hydraulic  
Leaking | Leak | Rinse Tank | Tightened | Not Mentioned  
Failed Sending | Open | Fuel Sender | None | NCV Harness  
No Oring | Hydraulic Leak | Boom | None | Tube  
Not Tighten | Fold Uneven | Auto Boom | None | Oring  
Out of Range | Getting Fault Code | Condenser | None | Sensor  
Lubricant Drip Drown | Not Working | Left-Air Duct | None | Counter  
Fault | Error Codes | Bulkhead Connector | None | Threads  
Internal Issue | Product Leak | Braided Steel | None | Left Air Duct  
Screwed in a Thread | Does not Light | Intrip Unlocks | None | Compressor Line  
Faulty | None | Sensor | None | Intrip Unlocks  
None | None | Bolts | None |  
None | None | Harness | None |  
None | None | Pipe Fitting | None |  
None | None | Bulkhead Connector | None |  
None | None | SVM Sign | None |  
None | None | ELB | None |  

=== EXAMPLE WITH MULTIPLE ISSUES ===

Complaint: Fuel door will not stay open and there is an oil leak
Cause: GAS STRUT NOT INSTALLED AND OIL LINE LOOSE
Correction: INSTALL GAS STRUT AND TIGHTEN OIL LINE

Step-by-step reasoning:  
- First issue: Door doesn't stay open due to missing gas strut
- Second issue: Oil leak due to loose oil line
- Multiple fixes needed: Install gas strut and tighten oil line

Final Output:  
Root Cause: Not Installed, Loosened
Symptom Condition 1: Won't stay open
Symptom Component 1: Fuel Door
Symptom Condition 2: Oil Leak
Symptom Component 2: Oil Line
Symptom Condition 3: None
Symptom Component 3: None
Fix Condition 1: Installed
Fix Component 1: Gas Strut
Fix Condition 2: Tightened
Fix Component 2: Oil Line
Fix Condition 3: None
Fix Component 3: None
Estimated Time to Resolve: 2 hours


=== EXAMPLE 2: THREE ISSUES ===

Complaint: Boom not folding properly, hydraulic leak, and error codes showing
Cause: AUTO BOOM NOT TIGHTENED, HYDRAULIC LEAK IN BOOM, AND FAULTY SENSOR
Correction: TIGHTEN AUTO BOOM, REPAIR HYDRAULIC LEAK, AND REPLACE SENSOR

Step-by-step reasoning:  
- First issue: Boom folding problem due to loose auto boom
- Second issue: Hydraulic leak in the boom area
- Third issue: Error codes due to faulty sensor
- Multiple fixes needed: Tighten boom, repair hydraulic system, and replace sensor

Final Output:  
Root Cause: Not Tighten, Leaking, Faulty
Symptom Condition 1: Fold Uneven
Symptom Component 1: Auto Boom
Symptom Condition 2: Hydraulic Leak
Symptom Component 2: Boom
Symptom Condition 3: Error Codes
Symptom Component 3: Sensor
Fix Condition 1: Tightened
Fix Component 1: Auto Boom
Fix Condition 2: Repaired
Fix Component 2: Hydraulic
Fix Condition 3: Replaced
Fix Component 3: Sensor
Estimated Time to Resolve: 4 hours

=== EXAMPLE 3: TWO ISSUES WITH COMPLEX FIXES ===

Complaint: Compressor pressure line crushed and oil dripping from coupler
Cause: COMPRESSOR LINE DAMAGED AND COUPLER OUT OF FITTING
Correction: REPLACE COMPRESSOR LINE AND CLEAN/REINSTALL COUPLER

Step-by-step reasoning:  
- First issue: Crushed compressor line
- Second issue: Oil leak from coupler
- Complex fixes needed: Replace line and clean/reinstall coupler

Final Output:  
Root Cause: Not Mentioned, Out of Fitting
Symptom Condition 1: Crushed
Symptom Component 1: Compressor Pressure Line
Symptom Condition 2: Oil Dripping
Symptom Component 2: Coupler
Symptom Condition 3: None
Symptom Component 3: None
Fix Condition 1: Replaced
Fix Component 1: Braided Steel
Fix Condition 2: Cleaned Out
Fix Component 2: Coupler
Fix Condition 3: None
Fix Component 3: None
Estimated Time to Resolve: 3 hours



---
-There can be multiple issues,single issue in the complaint, cause, or correction so find accoridngly and genrate the final output .
Now classify the following:

Complaint: {complaint}  
Cause: {cause}  
Correction: {correction}

Step-by-step reasoning:  
[Let the model think aloud]

Final Output:  
Root Cause: [...]  
Symptom Condition 1: [...]  
Symptom Component 1: [...]  
Symptom Condition 2: [...]  
Symptom Component 2: [...]  
Symptom Condition 3: [...]  
Symptom Component 3: [...]  
Fix Condition 1: [...]  
Fix Component 1: [...]  
Fix Condition 2: [...]  
Fix Component 2: [...]  
Fix Condition 3: [...]  
Fix Component 3: [...]  
Estimated Time to Resolve: [...] hours
"""
# Function to call OpenAI's chat completions API
def get_tags_and_time(complaint, cause, correction):
    prompt = generate_prompt(complaint, cause, correction)
    response = openai.ChatCompletion.create(
        model="gpt-4.1-mini-2025-04-14",
        messages=[
            {"role": "system", "content": "You are a service data analyst."},
            {"role": "user", "content": prompt},
        ],
        temperature=0,
        api_key=API_Key,
        response_format=response_schema
        
    )
 
    # Extracting and parsing the output from the response
    output_text = response['choices'][0]['message']['content']

    # Parse it to a Python dictionary
    try:
        output = json.loads(output_text)
    except json.JSONDecodeError:
        raise ValueError("The model response is not valid JSON")

    return output

def extract_fields(json_output):
    def pad_list(lst, length=3):
        return lst + [None] * (length - len(lst))

    root_cause = ", ".join(json_output.get("root_cause", ["None"]))

    symptom_conditions = pad_list(json_output.get("symptom_conditions", ["None"]))
    symptom_components = pad_list(json_output.get("symptom_components", ["None"]))
    fix_conditions = pad_list(json_output.get("fix_conditions", ["None"]))
    fix_components = pad_list(json_output.get("fix_components", ["None"]))

    return {
        "Root Cause": root_cause,
        "Symptom Condition 1": symptom_conditions[0],
        "Symptom Component 1": symptom_components[0],
        "Symptom Condition 2": symptom_conditions[1],
        "Symptom Component 2": symptom_components[1],
        "Symptom Condition 3": symptom_conditions[2],
        "Symptom Component 3": symptom_components[2],
        "Fix Condition 1": fix_conditions[0],
        "Fix Component 1": fix_components[0],
        "Fix Condition 2": fix_conditions[1],
        "Fix Component 2": fix_components[1],
        "Fix Condition 3": fix_conditions[2],
        "Fix Component 3": fix_components[2],
        "Estimated Time to Resolve": str(json_output.get("estimated_time_to_resolve_hours", "None"))
    }



def process_dataframe(df):
    results = []
    original_columns = df.columns.tolist()
    
    for _, row in df.iterrows():
        complaint, cause, correction = row['Complaint'], row['Cause'], row['Correction']
        try:
            model_output_dict = get_tags_and_time(complaint, cause, correction)  # already a dict
            extracted = extract_fields(model_output_dict)
            combined = {col: row[col] for col in original_columns}
            combined.update(extracted)
            results.append(combined)
        except Exception as e:
            print(f"Error at row: {row.get('Primary Key', 'Unknown')} - {e}")
            continue

    return pd.DataFrame(results)
# Run & save
final_df = process_dataframe(df)
final_df = final_df.fillna("None")
output_path = "output.xlsx"
final_df.to_excel(output_path, index=False)